# Compute metrics for different runs and plot them
##### author: Elizabeth A. Barnes, Randal J. Barnes and Mark DeMaria
##### version: v0.1.0
##### date: 14 December 2021

```
conda create --name env-hurr-tfp python=3.9
conda activate env-hurr-tfp
pip install tensorflow==2.7.0
pip install tensorflow-probability==0.15.0
pip install --upgrade numpy scipy pandas statsmodels matplotlib seaborn 
pip install --upgrade palettable progressbar2 tabulate icecream flake8
pip install --upgrade keras-tuner sklearn
pip install --upgrade jupyterlab black isort jupyterlab_code_formatter
pip install silence-tensorflow
pip install tqdm
```

Use the command
```python -m pip freeze > requirements.txt```
to make a pip installation list.

In [1]:
import datetime
import os
import pickle
import pprint
import time

import experiment_settings
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import shash
from build_data import build_hurricane_data
from build_model import build_shash_model, build_bnn_model
import model_diagnostics
from silence_tensorflow import silence_tensorflow
import prediction
from sklearn.neighbors import KernelDensity
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

silence_tensorflow()
dpiFig = 400

In [2]:
__author__ = "Randal J Barnes and Elizabeth A. Barnes"
__version__ = "17 January 2022"

EXP_NAME_LIST = (
                 "intensity8_AL96",
                 "intensity9_AL96",
                 "intensity10_EPCP96",
                 "intensity11_EPCP96",
                 "intensity0_AL72",
                 "intensity1_AL72",
                 "intensity4_EPCP72",
                 "intensity5_EPCP72",
                 "intensity12_AL48",
                 "intensity13_AL48",    
                 "intensity2_EPCP48",
                 "intensity3_EPCP48",
                )
APPEND_NAME = ''

# EXP_NAME_LIST = (
                 # "intensity204_AL72",
                 # "intensity205_AL72",
                 # "intensity202_EPCP72",
                 # "intensity203_EPCP72",            
                 # "intensity206_AL48",
                 # "intensity207_AL48",
                 # "intensity200_EPCP48",
                 # "intensity201_EPCP48",    
                # )
# APPEND_NAME = '_clusterExtrapolation'



OVERWRITE_METRICS = True
DATA_PATH = "data/"
MODEL_PATH = "saved_models/"
METRIC_PATH = "model_metrics/"
FIGURE_PATH = "figures/summary_plots/"

In [3]:
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

## Define get_metrics()

In [4]:
def get_metrics():
    tf.random.set_seed(network_seed)
    shash_incs = np.arange(-160,161,1)

    if settings["uncertainty_type"] == "bnn":       
        # loop through runs for bnn calculation    
        runs = 5_000
        bins_plot = np.linspace(np.min(shash_incs), np.max(shash_incs), 1000)
        bnn_cpd = np.zeros((np.shape(x_eval)[0],runs))
        bnn_mode = np.zeros((np.shape(x_eval)[0],))

        for i in tqdm(range(0,runs)):
            bnn_cpd[:,i] = np.reshape(model_bnn.predict(x_eval),np.shape(bnn_cpd)[0])
        bnn_mean = np.mean(bnn_cpd,axis=1)
        bnn_median = np.median(bnn_cpd,axis=1)

        for j in tqdm(range(0,np.shape(bnn_mode)[0])):
            kde = KernelDensity(kernel="gaussian", bandwidth=4.).fit(bnn_cpd[j,:].reshape(-1,1))
            log_dens = kde.score_samples(bins_plot.reshape(-1,1))
            i = np.argmax(log_dens)
            bnn_mode[j] = bins_plot[i]

        mean_error, median_error, mode_error = model_diagnostics.compute_errors(onehot_eval, bnn_mean, bnn_median, bnn_mode)         
        bins, hist_bnn, pit_D, EDp_bnn = model_diagnostics.compute_pit('bnn',onehot_eval, bnn_cpd)
        iqr_capture = model_diagnostics.compute_interquartile_capture('bnn',onehot_eval, bnn_cpd)
        iqr_error_spearman, iqr_error_pearson = model_diagnostics.compute_iqr_error_corr('bnn',
                                                                                          onehot_val=onehot_eval, 
                                                                                          bnn_cpd=bnn_cpd, 
                                                                                          pred_median=bnn_median,
                                                                                         )
        
    else:        
        shash_cpd = np.zeros((np.shape(x_eval)[0],len(shash_incs)))
        shash_mean = np.zeros((np.shape(x_eval)[0],))
        shash_med = np.zeros((np.shape(x_eval)[0],))
        shash_mode = np.zeros((np.shape(x_eval)[0],))

        # loop through samples for shash calculation and get PDF for each sample
        for j in tqdm(range(0,np.shape(shash_cpd)[0])):
            mu_pred, sigma_pred, gamma_pred, tau_pred = prediction.params( x_eval[np.newaxis,j], model_shash )
            shash_cpd[j,:] = shash.prob(shash_incs, mu_pred, sigma_pred, gamma_pred, tau_pred)    
            shash_mean[j]  = shash.mean(mu_pred,sigma_pred,gamma_pred,tau_pred)#np.sum(shash_cpd[j,:]*shash_incs)
            shash_med[j]   = shash.median(mu_pred,sigma_pred,gamma_pred,tau_pred)

            i = np.argmax(shash_cpd[j,:])
            shash_mode[j]  = shash_incs[i]

        mean_error, median_error, mode_error = model_diagnostics.compute_errors(onehot_eval, shash_mean, shash_med, shash_mode)    
        bins, hist_shash, pit_D, EDp_shash = model_diagnostics.compute_pit('shash',onehot_eval, x_val=x_eval,model_shash=model_shash)
        iqr_capture = model_diagnostics.compute_interquartile_capture('shash',onehot_eval, x_val=x_eval,model_shash=model_shash)
        iqr_error_spearman, iqr_error_pearson = model_diagnostics.compute_iqr_error_corr('shash',
                                                                                                onehot_val=onehot_eval,
                                                                                                pred_median=shash_med,
                                                                                                x_val=x_eval,
                                                                                                model_shash=model_shash,
                                                                                               )

    # by definition Consensus is a correction of zero
    cons_error = np.mean(np.abs(0.0 - onehot_eval[:,0]))
        
    return mean_error,median_error,mode_error,pit_D,iqr_capture,cons_error,iqr_error_spearman,iqr_error_pearson


## Evaluate the models

In [5]:
# raise ValueError('do not compute metrics yet')

In [6]:
import imp
imp.reload(model_diagnostics)

for exp_name in EXP_NAME_LIST:
    settings = experiment_settings.get_settings(exp_name)
    RNG_SEED_LIST = np.copy(settings['rng_seed'])

    for rng_seed in RNG_SEED_LIST:
        settings['rng_seed'] = rng_seed
        NETWORK_SEED_LIST = [settings["rng_seed"]]
        network_seed = NETWORK_SEED_LIST[0]
        tf.random.set_seed(network_seed)  # This sets the global random seed.    

        #----------------------------------------------------------------------------------------------------
        # get the data
        (
            x_train,
            onehot_train,
            x_val,
            onehot_val,
            x_eval,
            onehot_eval,    
            data_summary,
            df_val,
            df_eval,
        ) = build_hurricane_data(DATA_PATH, settings, verbose=0)

        #----------------------------------------------------------------------------------------------------
        # get the model
        if settings["uncertainty_type"] == "bnn":       
            model_name_bnn = (
                exp_name + "_" + settings["uncertainty_type"] + '_' + f"network_seed_{network_seed}_rng_seed_{settings['rng_seed']}"
            )
            model_bnn = build_bnn_model(
                x_train,
                onehot_train,
                hiddens=settings["hiddens"],
                output_shape=onehot_train.shape[1],
                ridge_penalty=settings["ridge_param"],
                act_fun=settings["act_fun"],
            )
            try:
                model_bnn.load_weights(MODEL_PATH + model_name_bnn + "_weights.h5")
            except:
                print('tf model does not exist. skipping...')
                continue
            model_name = model_name_bnn

        elif settings["uncertainty_type"][:5] == "shash": 
            model_name_shash = (
                exp_name + "_" + settings["uncertainty_type"] + '_' + f"network_seed_{network_seed}_rng_seed_{settings['rng_seed']}"
            )
            model_shash = build_shash_model(
                x_train,
                onehot_train,
                hiddens=settings["hiddens"],
                output_shape=onehot_train.shape[1],
                ridge_penalty=settings["ridge_param"],
                act_fun=settings["act_fun"],
            )
            try:
                model_shash.load_weights(MODEL_PATH + model_name_shash + "_weights.h5")
            except:
                print('tf model does not exist. skipping...')
                continue
            model_name = model_name_shash

        #----------------------------------------------------------------------------------------------------
        # check if the metric filename exists already
        metric_filename = METRIC_PATH + model_name + '_metrics.pkl'
        if (os.path.exists(metric_filename) and OVERWRITE_METRICS==False):
            # print(metric_filename + ' exists. Skipping...')
            continue
            
        # get metrics and put into a dictionary
        pprint.pprint(model_name)
        
        # if running out of cluster comparison, grab only the cluster that was not seen during training
        if(settings["train_condition"]=='cluster'):
            print('grabbing out of sample ' + 'CLUSTER')
            i_index = np.where(data_summary["cluster_eval"] == data_summary["cluster_out"])[0]
            x_eval = x_eval[i_index,:]
            onehot_eval = onehot_eval[i_index,:]
            df_eval.iloc[i_index]       
        
        # compute the metrics
        mean_error, median_error, mode_error, pit_D, iqr_capture, cons_error, iqr_error_spearman,iqr_error_pearson = get_metrics()        
        d = {'uncertainty_type': settings["uncertainty_type"],
             'network_seed': network_seed,
             'rng_seed': settings['rng_seed'],
             'exp_name': exp_name,
             'mean_error': mean_error,
             'median_error': median_error,
             'mode_error': mode_error,
             'cons_error': cons_error,
             'pit_d': pit_D,
             'iqr_capture': iqr_capture,
             'iqr_error_spearman': iqr_error_spearman[0],
             'iqr_error_pearson': iqr_error_pearson[0],             
             'iqr_error_spearman_p': iqr_error_spearman[1],
             'iqr_error_pearson_p': iqr_error_pearson[1],             
        }
        pprint.pprint(d, width=80)  
        df = pd.DataFrame(data=d, index=[0])
        df.to_pickle(metric_filename)

'intensity8_AL96_shash3_network_seed_605_rng_seed_605'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:03<00:00, 76.05it/s]


{'cons_error': 13.256,
 'exp_name': 'intensity8_AL96',
 'iqr_capture': 0.56,
 'iqr_error_pearson': 0.07677067460018999,
 'iqr_error_pearson_p': 0.18480199042844817,
 'iqr_error_spearman': -0.0037178203261661615,
 'iqr_error_spearman_p': 0.9488699535645244,
 'mean_error': 13.912250697463751,
 'median_error': 13.431230440785487,
 'mode_error': 13.608666662524143,
 'network_seed': 605,
 'pit_d': 0.025819870350582474,
 'rng_seed': 605,
 'uncertainty_type': 'shash3'}
'intensity8_AL96_shash3_network_seed_122_rng_seed_122'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:03<00:00, 78.39it/s]


{'cons_error': 13.905667,
 'exp_name': 'intensity8_AL96',
 'iqr_capture': 0.4266666666666667,
 'iqr_error_pearson': 0.44987226593775764,
 'iqr_error_pearson_p': 2.3607345387098984e-16,
 'iqr_error_spearman': 0.45222500560072587,
 'iqr_error_spearman_p': 1.5791867161828154e-16,
 'mean_error': 14.377890497843424,
 'median_error': 13.974807382424672,
 'mode_error': 14.21166668454806,
 'network_seed': 122,
 'pit_d': 0.017448326213974033,
 'rng_seed': 122,
 'uncertainty_type': 'shash3'}
'intensity8_AL96_shash3_network_seed_786_rng_seed_786'


...

{'cons_error': 13.412695,
 'exp_name': 'intensity3_EPCP48',
 'iqr_capture': 0.517578125,
 'iqr_error_pearson': 0.3282844869273568,
 'iqr_error_pearson_p': 2.4913445847430406e-14,
 'iqr_error_spearman': 0.3276072422780696,
 'iqr_error_spearman_p': 2.834141441825721e-14,
 'mean_error': 12.297815480867218,
 'median_error': 12.312497654791514,
 'mode_error': 12.460978384212266,
 'network_seed': 331,
 'pit_d': 0.013412722432335466,
 'rng_seed': 331,
 'uncertainty_type': 'bnn'}
'intensity3_EPCP48_bnn_network_seed_908_rng_seed_908'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:35<00:00, 14.29it/s]


{'cons_error': 13.402929,
 'exp_name': 'intensity3_EPCP48',
 'iqr_capture': 0.529296875,
 'iqr_error_pearson': 0.31275789013369737,
 'iqr_error_pearson_p': 4.4188124110766e-13,
 'iqr_error_spearman': 0.33913291719595795,
 'iqr_error_spearman_p': 3.0228288167823958e-15,
 'mean_error': 12.590265611913328,
 'median_error': 12.60427193778014,
 'mode_error': 12.75728248607324,
 'network_seed': 908,
 'pit_d': 0.01675139190811841,
 'rng_seed': 908,
 'uncertainty_type': 'bnn'}
'intensity3_EPCP48_bnn_network_seed_444_rng_seed_444'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [00:35<00:00, 14.25it/s]


{'cons_error': 14.137499,
 'exp_name': 'intensity3_EPCP48',
 'iqr_capture': 0.53125,
 'iqr_error_pearson': 0.3239406579551075,
 'iqr_error_pearson_p': 5.663968085995632e-14,
 'iqr_error_spearman': 0.320704461639258,
 'iqr_error_spearman_p': 1.0355050734120458e-13,
 'mean_error': 12.735750582085496,
 'median_error': 12.723766089358833,
 'mode_error': 12.712663480789265,
 'network_seed': 444,
 'pit_d': 0.014320549046737002,
 'rng_seed': 444,
 'uncertainty_type': 'bnn'}


In [7]:
raise ValueError('do not plot yet')

ValueError: do not plot yet

In [ ]:
df_metrics = pd.DataFrame()

for exp_name in EXP_NAME_LIST:
    settings = experiment_settings.get_settings(exp_name)
    RNG_SEED_LIST = np.copy(settings['rng_seed'])
    
    for rng_seed in RNG_SEED_LIST:
        settings['rng_seed'] = rng_seed
        NETWORK_SEED_LIST = [settings["rng_seed"]]
        network_seed = NETWORK_SEED_LIST[0]
    
        model_name = (
                exp_name + "_" + settings["uncertainty_type"] + '_' + f"network_seed_{network_seed}_rng_seed_{settings['rng_seed']}"
        )
        
        # load the metric filename
        metric_filename = METRIC_PATH + model_name + '_metrics.pkl'
        if os.path.exists(metric_filename)==False:
            print(metric_filename + ' DOES NOT exist. Skipping...')
            continue
    
        # pprint.pprint(model_name)
        df = pd.read_pickle(metric_filename)
        df['basin_lead'] = exp_name[exp_name.rfind('_')+1:]
        df['mean_error_reduction'] = df['cons_error']-df['mean_error']
        df['median_error_reduction'] = df['cons_error']-df['median_error']        
        df['mode_error_reduction'] = df['cons_error']-df['mode_error']        
        df_metrics = pd.concat([df_metrics,df])
df_metrics

In [ ]:
raise ValueError('do not plot yet')

In [ ]:
import seaborn as sns
f, axs = plt.subplots(4, 2, figsize=(15,20))
axs = axs.flatten()
colors = ('#D95980','#284E60')#('#E1A730','#284E60')

for imetric, metric in enumerate(('mean_error',
                                  'mean_error_reduction',
                                  'median_error',
                                  'median_error_reduction',                                  
                                  'mode_error',
                                  'mode_error_reduction',                                  
                                  'pit_d',
                                  'iqr_capture')):
    ax = axs[imetric]
    g1 = sns.boxplot(x='basin_lead', 
                     y=metric, 
                     hue="uncertainty_type",
                     data=df_metrics,
                     palette=colors,
                     boxprops={'alpha':.2,
                               'edgecolor': 'white',
                              },
                     fliersize=0,
                     ax=ax)
    g2 = sns.swarmplot(x="basin_lead", 
                       y=metric, 
                       hue="uncertainty_type",
                       palette=colors,
                       data=df_metrics, 
                       dodge=True,
                       ax=ax)
    
    if(metric=='iqr_capture'):
        ax.axhline(y=0.5,linewidth=3,linestyle='--',color='gray')
        ax.set_ylim(0,1.0)
    if(metric=='pit_d'):
        ax.set_ylim(0,None)
    if(metric.find('reduction') > -1):
        ax.axhline(y=0.0,linewidth=3,linestyle='--',color='gray')
        ax.set_ylim(-4.,4.)
    if(metric.find('reduction') == -1 and metric.find('error') > -1):
        ax.set_ylim(0.,22.)

    
    ax.set_title(metric + APPEND_NAME)
    ax.legend(fontsize=10,frameon=True)

plt.savefig(FIGURE_PATH + 'comparisonsMetrics' + APPEND_NAME + '.png', dpi=dpiFig)    
plt.tight_layout()
plt.show()

In [ ]:
# import seaborn as sns
# f, axs = plt.subplots(4, 2, figsize=(15,50))
# axs = axs.flatten()

# for imetric, metric in enumerate(('mean_error',
#                                   'mean_improvement',
#                                   'median_error',
#                                   'median_improvement',                                  
#                                   'mode_error',
#                                   'mode_improvement',                                  
#                                   'pit_d',
#                                   'iqr_capture')):
#     ax = axs[imetric]
#     g1 = sns.boxplot(x='exp_name', 
#                      y=metric, 
#                      hue="uncertainty_type",
#                      data=df_metrics,
#                      boxprops=dict(alpha=.4),
#                      ax=ax)
#     g2 = sns.swarmplot(x="exp_name", 
#                        y=metric, 
#                        hue="uncertainty_type", 
#                        data=df_metrics, 
#                        dodge=True,
#                        ax=ax)
    
#     if(metric=='iqr_capture'):
#         ax.axhline(y=0.5,linewidth=3,linestyle='--',color='gray')
#     if(metric=='pit_d'):
#         ax.set_ylim(0,None)
#     if(metric.find('improvement') > -1):
#         ax.axhline(y=0.0,linewidth=3,linestyle='--',color='gray')
    
#     ax.set_title(metric)
#     ax.legend(fontsize=10,frameon=True)
#     ax.set_xticklabels(ax.get_xticklabels(),rotation = 30)

# # plt.savefig(FIGURE_PATH + 'comparisonsMetrics.png', dpi=dpiFig)    
# plt.tight_layout()
# plt.show()